# Проект №3. О вкусной и здоровой пище 
**[SF-DST-10] Restaurant Rating prediction Sergey Kuzmenko**

# import

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import pandas as pd
import numpy as np 
import re
import json
import datetime as dt
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
import unicodedata #для транслитерации не ASCII
import unidecode #для транслитерации не ASCII 
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
import itertools
%matplotlib inline


import os
for dirname, _, filenames in os.walk('/kaggle'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

ModuleNotFoundError: No module named 'unidecode'

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

In [ ]:
# Для воспроизводимости результатов зададим:
# - общий параметр для генерации случайных чисел
RANDOM_SEED = 20726
# - общую текущую дату
CURRENT_DATE = pd.to_datetime('22/02/2020')


# DATA

In [ ]:
DATA_DIR = '/kaggle/input/sf-dst-restaurant-rating/'
df_in = pd.read_csv(DATA_DIR+'main_task.csv')
# Путь к датасету для которого требуется предсказать рейтинг
df_kagle = pd.read_csv(DATA_DIR+'kaggle_task.csv')
#Путь к 'submission.csv'
submission_path = pd.read_csv(DATA_DIR+'sample_submission.csv')

In [ ]:
DS_IMPORT_SC_DIR = '/kaggle/input/cities-and-countries/'
# Путь к датасету стран - отсюда берем столицы
df_county = pd.read_csv(DS_IMPORT_SC_DIR+'country-list.csv')
#датасет по городам и странам
df_urb_xls = pd.read_excel(DS_IMPORT_SC_DIR+'global-city-population-estimates.xls', index_col=None, header=0, sheet_name = 'CITIES-OVER-300K')

# Путь к датасету, содержащему слова с позитивной окраской
df_pos_words = pd.read_csv('/kaggle/input/opinion-lexicon-english/positive-words.txt',skiprows=34, names=['word'])
pos_words_list = df_pos_words['word'].to_list() 

# FUNC

In [ ]:
 def get_season(date):
    '''
    Время года из даты
    '''
    if (pd.isna(date)):
        return "OTHER"
    month = date.month
    if (month > 11 or month <= 3):
       return "WINTER"
    elif (month == 4 or month == 5):
       return "SPRING"
    elif (month >=6 and month <= 9):
       return "SUMMER"
    else:
       return "FALL"

In [ ]:
def round_of_rating(number):
    """
    Округляем до 0.5
    """
    return np.round(number * 2) / 2

In [ ]:
def get_Weighed_Rank_RK(row):
    '''
    Вычисляем относительную позицию ресторана среди всех ресторанов города
    '''
    Weighed_Rank = row['Ranking'] / row['Restaurants Count']

    return Weighed_Rank

In [ ]:
def get_Weighed_Rank(row):
    i=0
    city_min = CityMinMax[CityMinMax['City'] == row.City ]['min'].iloc[0]
    city_max = CityMinMax[CityMinMax['City'] == row.City ]['max'].iloc[0]
    Weighed_Rank = round(1-(row['Ranking'] - city_min)/(city_max - city_min),3)
    #print('<---',Weighed_Rank,'--->')
    return Weighed_Rank

In [ ]:
def rev_time_delta(reviews):
    '''
    Вычисляем время между review в днях
    '''
    if (pd.notna(reviews)):
        reviews_dt_list = get_reviews(reviews)['reviews_dt']
        if reviews_dt_list:
            return (max(reviews_dt_list) - min(reviews_dt_list)).days
        else:
            return dt.timedelta(days=3650).days
    else:
        return dt.timedelta(days=3650).days

In [ ]:
def get_reviews(rev):
    '''
    Получаем review в виде:
    review['reviews_txt'][1] - list of reviews
    review['reviews_dt'][1] - list of reviews dates
    '''
    if  not pd.isna(rev): 
        rev = str(rev).replace("'",'"')
        rev = rev.replace('], [', '], "reviews_dt": [')
        rev = '{ "reviews_txt":' + rev + '}'
        rev = rev.replace('[[','[').replace(']]',']')
        d = json.loads(rev)

        d['reviews_dt'] = [dt.datetime.strptime(date, '%m/%d/%Y').date() if len(date.split('/')[2])==4 else dt.datetime.strptime(date, '%m/%d/%y').date() for date in d['reviews_dt']]
        return d
    else:
        return {}

In [ ]:
def get_cuisines(cuisines):
    '''
    Получаем список кухонь в виде:
    cuisines[0] - list of cusines
    если был NaN, то возвращается 'Regionl Cusine' -как самая популярная в регионе/городе/стране
    '''
    if cuisines == 'NaN': return ['Regionl Cusine']#['Vegetarian Friendly'] # 'Vegetarian Friendly' - самая популярная
    if  cuisines:
        cuisines = str(cuisines).replace("'",'"')
        return json.loads(cuisines)
    else:
        return ['Regionl Cusine'] #return ['Vegetarian Friendly']

In [ ]:
allCusines = []
def cuisine_styles_count(row):
    '''
    Получаем количество кухонь
    '''
    global allCusines
    cusines = get_cuisines(row['Cuisine Style'])
    
    if row['Cuisine Style'] != 'NaN':    
        cusines = get_cuisines(row['Cuisine Style'])
        allCusines.extend(cusines)
        cuisines_count =len(cusines)
    else:
        cuisines_count = 1

    return cuisines_count

In [ ]:
def cleanup_string(str_in):
    '''
    "Чистим" текст в review для последующй десериализации
    Пока текст в анализе не используется, не успел... 
    '''
    try:      
        
        #middle

        str = str_in.replace("', \"","⅞").replace("', '","⅞").replace("\", '","⅞").replace("\", \"","⅞")# ", 
        str = str.replace("\", \"\"","⅞").replace("\"\", '","⅞").replace("\", \'","⅞").replace("\"\", \'","⅞")
        str = str.replace("\', \'","⅞")
        #middle
        #left
        str = str.replace("[['","≤").replace("['","⅛")
        #left
        #right
        str = str.replace("']]","≥").replace("']","⅝")
        #right
        #cleanups
        str = str.replace('\'', ' ').replace('\"', ' ').replace('\'', ' ').replace('"', ' ')     
        str = str.replace("\\", " ").replace("[[`", "≤").replace('\'"', '\'').replace('\'\"', '\'')
        str = str.replace('"\'', '\'').replace('\"\'', '\'').replace("[''" ,"≤").replace("[\'\'" ,"≤")
        str = str.replace('\'', ' ').replace('\"', ' ')
        str = str.replace('\'', ' ').replace('\"', ' ')
        str = str.replace('\'', ' ').replace('"', ' ')
        #cleanups
        #middle
        str = str.replace("⅞", "', '")
        #middle
        #left
        str = str.replace("≤", "[['").replace("⅛", "['").replace('[[ ', '[[ \'')
        #left
        #right
        str = str.replace("≥" ,"']]").replace("⅝", "']").replace(' ]', ' \']')
        str = str.replace(', nan]', '\', \'nan\']').replace('[nan, ', '[\'nan\', \'')
        #right
    except Exception:
        print('<----',str_in,'---->')
    return str

In [ ]:
def get_city_population_and_country():
    '''
    Получаем популяцию по городам, а так же ISO код страны по городу (из внешних источников)
    '''
    population_city_dict = {}
    country_city_dict = {}
    cities = df['City'].unique()
    # Чистим названия городов от Unicode символов - транслитерацией
    df_urb_xls['Urban Agglomeration TL'] = df_urb_xls['Urban Agglomeration'].apply(lambda s: ''.join((c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')) )
    df_urb_xls['Urban Agglomeration TL'] =  df_urb_xls['Urban Agglomeration'].apply(lambda x: unidecode.unidecode(x))

    for city in cities:
        vals = df_urb_xls[df_urb_xls['Urban Agglomeration TL'].str.contains(city)]['2015'].max()/1000
        vals = 0.3 if pd.isna(vals) else vals # т.к. в датасете города от 300тыс
        population_city_dict[city] = vals
        country = df_urb_xls[df_urb_xls['Urban Agglomeration TL'].str.contains(city)]['Country Code']
        country = -1 if country.shape[0] < 1 else country.values[0]
        country_city_dict[city] = country

    population_city_dict['Luxembourg'] = 0.613894 
    population_city_dict['Brussels'] = 2.115468 
    population_city_dict['Geneva'] = 0.686562 
    population_city_dict['Oporto'] = 0.214349 
    population_city_dict['Ljubljana'] = 0.279631



    country_city_dict['Luxembourg'] = 442 
    country_city_dict['Brussels'] = 56 
    country_city_dict['Geneva'] = 756 
    country_city_dict['Oporto'] = 620 
    country_city_dict['Ljubljana'] = 705
    
    #Видимо, эти данные корректнее???
    population_city_dict = {    'London': 8.173900,
    'Paris': 2.240621,
    'Madrid': 3.155360,
    'Barcelona': 1.593075,
    'Berlin': 3.326002,
    'Milan': 1.331586,
    'Rome': 2.870493,
    'Prague': 1.272690,
    'Lisbon': .547733,
    'Vienna': 1.765649,
    'Amsterdam': .825080,
    'Brussels': .144784,
    'Hamburg': 1.718187,
    'Munich': 1.364920,
    'Lyon': .496343,
    'Stockholm': 1.981263,
    'Budapest': 1.744665,
    'Warsaw': 1.720398,
    'Dublin': .506211 ,
    'Copenhagen': 1.246611,
    'Athens': 3.168846,
    'Edinburgh': .476100,
    'Zurich': .402275,
    'Oporto': .221800,
    'Geneva': .196150,
    'Krakow': .756183,
    'Oslo': .673469,
    'Helsinki': .574579,
    'Bratislava': .413192,
    'Luxembourg': .576249,
    'Ljubljana': .277554
    }
    return population_city_dict, country_city_dict

In [ ]:
def get_capital_city_dict():
    '''
    Возвращаем словарь город == столица или нет
    '''
    capital_city_dict = {}
    cities =df['City'].unique()
    df_county['capital TL'] = df_county['capital'].apply(lambda s: ''.join((c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')) )
    df_county['capital TL'] =  df_county['capital'].apply(lambda x: unidecode.unidecode(x))
    for city in cities:
        vals = df_county[df_county['capital TL'].str.contains(city)]['country'].shape[0]   
        vals = 0 if pd.isna(vals) or vals==0 else 1 
        capital_city_dict[city] = vals
    capital_city_dict['Barcelona'] = 0 # Страна Басков != Испания, дадим Барселоне статус мтолицы, но оставим в Испании ! 0 и не надо врать :)
    capital_city_dict['Zurich'] = 1 
    capital_city_dict['Geneva'] = 1 
    capital_city_dict['Oporto'] = 1 
    return capital_city_dict

In [ ]:
def is_cuisine_top_N(cs):
    '''
    Возвращаем список кухонь, входящих в основной список кухонь,для остальных Other
    '''
    c = get_cuisines(cs)
    c = set(c)

    shared_cousines=()
    shared_cousines=c.intersection(topNcusines)

    if len(shared_cousines) != len(c):
        shared_cousines = list(shared_cousines)
        shared_cousines.extend(['Other'])

    return list(shared_cousines)

In [ ]:
def createWordList(line):
    wordList2 =[]
    wordList1 = line.split()
    for word in wordList1:
        cleanWord = ""
        for char in word:
            if char in '!,.?":;0123456789':
                char = ""
            cleanWord += char
        wordList2.append(cleanWord.lower())
    return wordList2

In [ ]:
def count_positive_words_proportion(reviews):
    '''
    Количество позитивных слов в приведенных отзывах
    '''
    pos_words_count = 0
    txts=get_reviews(reviews)['reviews_txt']
    txt = ' '.join(txts)
    #print(type(txt))
    words = createWordList(txt)
    
    words_count = len(words) if len(words) > 0 else 1
    words_count = 1
    pos_words_in_review=set(words).intersection(pos_words_list)
    for word in words:
        if word in pos_words_list:
            #print(word)
            pos_words_count +=1  
    return np.round(pos_words_count/words_count,2)

In [ ]:
def list_positive_words(reviews): 
    '''
    Список уникальных позитивных слов в приведенных отзывах
    '''
    txts=get_reviews(reviews)['reviews_txt']
    txt = ' '.join(txts)
    #print(type(txt))
    words = createWordList(txt)
    
    words_count = len(words) if len(words) > 0 else 1
    words_count = 1
    pos_words_in_review=set(words).intersection(pos_words_list)
    #print(len(pos_words_in_review))
    if (len(pos_words_in_review) == 0):
        return np.NAN
    else:
        return list(pos_words_in_review)

---  

# WORK

In [ ]:
df_in.info()

In [ ]:
df_in.head(5)

In [ ]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет

df_kagle['ForTrain'] = False # помечаем где у нас тест
df_kagle['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

df_in['ForTrain'] = True # помечаем где у нас трейн
df_in = df_in.append(df_kagle, sort=False).reset_index(drop=True)# объединяем

Подробнее по признакам:
* City: Город 
* Cuisine Style: Кухня
* Ranking: Ранг ресторана относительно других ресторанов в этом городе
* Price Range: Цены в ресторане в 3 категориях
* Number of Reviews: Количество отзывов
* Reviews: 2 последних отзыва и даты этих отзывов
* URL_TA: страница ресторана на 'www.tripadvisor.com' 
* ID_TA: ID ресторана в TripAdvisor
* Rating: Рейтинг ресторана

In [ ]:
# Выбираем нужные для последующего анализа столбцы
df = df_in[['Ranking', 'Rating', 'Number of Reviews', 'City', 'Price Range', 'Cuisine Style', 'Reviews', 'Restaurant_id', 'ID_TA', 'ForTrain' ]].copy()

In [ ]:
df.sample(5)

In [ ]:
df.Reviews[1]

Как видим, большинство признаков у нас требует очистки и предварительной обработки.

* ## 1. Обработка NAN и Обработка признаков
У наличия пропусков могут быть разные причины, но пропуски нужно либо заполнить, либо исключить из набора полностью. Но с пропусками нужно быть внимательным, **даже отсутствие информации может быть важным признаком!**   
По этому перед обработкой NAN лучше вынести информацию о наличии пропуска как отдельный признак 

**Посмотрим на признаки и к-во пропусков (NaN)**

In [ ]:
df.info()

**Посмотрим на признаки и к-во уникальных значений**

In [ ]:
df.nunique(dropna=False)

**Посмотрим на гистограммы числовых признаков**

In [ ]:
df[df.ForTrain][['Ranking', 'Rating', 'Number of Reviews','Price Range', 'Restaurant_id']].hist(figsize=(20, 10), bins=100);
plt.tight_layout()


In [ ]:
df[df.ForTrain]['Restaurant_id'].apply(lambda x: x.split('_')[1]).astype(int).hist(figsize=(10,5), bins=100);
plt.tight_layout()
# Restaurant_id Очень похож на Ranking

In [ ]:

df[df.ForTrain]['ID_TA'].apply(lambda x: x.replace('d','')).astype(int).hist(figsize=(10, 5), bins=100);
plt.tight_layout()
# Видно Несколько групп - на ID - точно не похоже. 

In [ ]:
df['ID_TA'] = df['ID_TA'].apply(lambda x: x[1:]) 

In [ ]:
# Reviews - убираем NaN и "причесываем" текст отзывов 
df['Reviews_txt_NaN'] = df['Reviews'].apply(lambda x: x ==  '[[], []]')

df['Reviews'] = df['Reviews'].fillna('[[], []]')
df['Reviews'] = df['Reviews'].apply(lambda x: cleanup_string(x))

In [ ]:
# Не будем кодировать названия городов - get_dummies справится 
# df["City"] = df["City"].astype('category')
# #df["City"] = df["City"].cat.codes
# encoder = LabelEncoder()
# df['City'] = encoder.fit_transform(df['City'])

In [ ]:
# Перекодируем Price Range и удаляем NaN
cleanup_nums = {'Price Range':     {"$": 1, "$$ - $$$": 2, "$$$$": 3, np.NaN: 2}} # чаще всего встречается "$$ - $$$" == 2
df['Price Range NAN'] = df['Price Range'].isna()
df.replace(cleanup_nums, inplace=True)

In [ ]:
# Получаем Cuisines Count, самую популярную кухню, среднее к-во кухонь в ресторане и устраняем NaN
df['Cuisine Style NAN'] = df['Cuisine Style'].isna()
df['Cuisine Style'] = df['Cuisine Style'].fillna('NaN')
df['Cuisines Count'] = df.apply(cuisine_styles_count, axis=1)

most_popular_cusine = pd.Series(allCusines).value_counts().index[0]
average_cousines_count = np.round(df['Cuisines Count'].mean())

In [ ]:
# Number of Reviews
df['Number of Reviews NAN'] = df['Number of Reviews'].isna()
replace_val = df['Number of Reviews'].mean()
replace_val = np.round(replace_val)
df['Number of Reviews'] = df['Number of Reviews'].fillna(replace_val)

In [ ]:
df.sample(5)

# EDA 
[Exploratory Data Analysis](https://ru.wikipedia.org/wiki/Разведочный_анализ_данных) - Анализ данных
На этом этапе мы строим графики, ищем закономерности, аномалии, выбросы или связи между признаками.
В общем цель этого этапа понять, что эти данные могут нам дать и как признаки могут быть взаимосвязаны между собой.
Понимание изначальных признаков позволит сгенерировать новые, более сильные и, тем самым, сделать нашу модель лучше.
![](https://miro.medium.com/max/2598/1*RXdMb7Uk6mGqWqPguHULaQ.png)

### Посмотрим распределение признака

In [ ]:
plt.rcParams['figure.figsize'] = (10,5)
df[df.ForTrain]['Ranking'].hist(bins=100);

У нас много ресторанов, которые не дотягивают и до 2500 места в своем городе, а что там по городам?

In [ ]:
df[df.ForTrain]['City'].value_counts(ascending=True).plot(kind='barh');

А кто-то говорил, что французы любят поесть=) Посмотрим, как изменится распределение в большом городе:

In [ ]:
df[df.ForTrain]['Ranking'][df[df.ForTrain]['City'] =='London'].hist(bins=100);

In [ ]:
# посмотрим на топ 10 городов
for x in (df[df.ForTrain]['City'].value_counts())[0:10].index:
    df[df.ForTrain]['Ranking'][df[df.ForTrain]['City'] == x].hist(bins=100)
plt.show()

Получается, что Ranking имеет нормальное распределение, просто в больших городах больше ресторанов, из-за мы этого имеем смещение.

>Подумайте как из этого можно сделать признак для вашей модели. Я покажу вам пример, как визуализация помогает находить взаимосвязи. А далее действуйте без подсказок =) 


### Посмотрим распределение целевой переменной

In [ ]:
df[df.ForTrain]['Rating'].value_counts(ascending=True).plot(kind='barh');

### Посмотрим распределение целевой переменной относительно признака

In [ ]:
df[df.ForTrain]['Ranking'][df[df.ForTrain]['Rating'] == 5].hist(bins=100);

In [ ]:
df[df.ForTrain]['Ranking'][df[df.ForTrain]['Rating'] < 4].hist(bins=100);

### Корреляция имеющихся признаков - практически единственным признаком, коррелирующим с Rating является Ranking. Он, в свою очередь, уже имеет слабую корреляцию практически со всеми признаками 

In [ ]:
plt.rcParams['figure.figsize'] = (15,15)
sns.heatmap(df[df.ForTrain].drop(['ForTrain'], axis=1).corr(), square=True,
            annot=True, fmt=".1f", linewidths=0.1, cmap="RdBu");
plt.tight_layout()

**Поищем другие/дополнительные признаки, привлечем внешние данные **

In [ ]:
population_city_dict = {}
country_city_dict = {}
# Получаем словари популяции по городам, а так же ISO код страны по городу
population_city_dict, country_city_dict = get_city_population_and_country()

In [ ]:
# Вычисляем страну для города в каждой строке
df['Country'] = df["City"].apply(lambda x: country_city_dict[x])

In [ ]:
# Вычисляем к-во ресторанов для города в каждой строке
restorants_in_city = df.groupby('City')['Ranking'].count().to_dict()
df['Restaurants Count'] = df['City'].map(restorants_in_city)

In [ ]:
# Вычисляем население (в тыс. чел) для города в каждой строке
df['Population'] = df["City"].map(population_city_dict)

In [ ]:
# Вычисляем к-во ресторанов на 1000 чел для города в каждой строке
df['Restaurants for Population'] = df['Restaurants Count'] / (df['Population']*1000) #

In [ ]:
# Вычисляем является ли город столицей в каждой строке
capital_city_dict = get_capital_city_dict()
df['isCapital'] = df["City"].map(capital_city_dict)

In [ ]:
# Получаем относительную позицию ресторана среди всех ресторанов города
df['Weighed Rank'] = df.apply(lambda x: get_Weighed_Rank_RK(x), axis=1)

CityMinMax = df.groupby('City')['Ranking'].agg([min,max])
CityMinMax =CityMinMax.reset_index()
df['Weighed Rank min max'] = df.apply(lambda x: get_Weighed_Rank(x), axis=1)

In [ ]:
# Флаги (1/0) isMostPopCusine - есть ли в ресторане самая популярная кухня; isMultyCusine - к-во кухонь в ресторане больше или столько же чем в среднем
df['isMostPopCusine'] = df['Cuisine Style'].apply(lambda x: 1 if most_popular_cusine in x else 0 )
df['isMultyCusine'] = df['Cuisines Count'].apply(lambda x: 1 if  x >= average_cousines_count else 0 )

In [ ]:
# RevTimeDelta - время между review в днях

df['RevTimeDelta'] = df['Reviews'].apply(rev_time_delta)

In [ ]:
# NewestReviewDate - время, прошедшее со момента последнего review до '22/02/2020'
df['NewestReviewDate'] = df['Reviews'].apply(lambda x: get_reviews(x)['reviews_dt'])
df['NewestReviewDate'] = df['NewestReviewDate'].apply(lambda x: sorted(x,reverse=True)[0] if len(x)!=0 else pd.NaT)
df['NewestReviewDate'] = df['NewestReviewDate'].fillna(dt.date(1970,1,1))
df['NewestReviewDate'] = df['NewestReviewDate'].apply(lambda x: (CURRENT_DATE.date()-x).total_seconds()//86400)

In [ ]:
df['NewestReviewSeason'] = df['Reviews'].apply(lambda x: get_reviews(x)['reviews_dt'])
df['NewestReviewSeason'] = df['NewestReviewSeason'].apply(lambda x: sorted(x,reverse=True)[0] if len(x)!=0 else pd.NaT)
#df['NewestReviewSeason'] = df['NewestReviewSeason'].fillna(dt.date(2020,2,22))
df['NewestReviewSeason'] = df['NewestReviewSeason'].apply(lambda x: get_season(x))

In [ ]:
#  'TxtReviewsCount' - к-во отзывов, не сильно улучшает результат, но пусть будут
df['TxtReviewsCount'] = df['Reviews'].apply(lambda x: len(get_reviews(x)['reviews_txt']))

In [ ]:
# К-во позитивных слов в представленных отзывах
df['PositiveWords'] = df['Reviews'].apply(lambda x: count_positive_words_proportion(x))

In [ ]:
# Список уникальных позитивных слов в представленных отзывах
df['PositiveWordsList'] = df['Reviews'].apply(lambda x: list_positive_words(x))

In [ ]:
cusines_in_city={}
cusines_count_in_city={}
for city_name, group in df.groupby('City'):

    cusines = group['Cuisine Style'].apply(get_cuisines)
   
    cusines_list = list(itertools.chain.from_iterable(cusines))
    #cusines_list = [x for x in cusines_list if x != 'Vegetarian Friendly'] # удаляем 'Vegetarian Friendly' - она "забивает" все результаты
    cusines_in_city[city_name] = Counter(cusines_list)

for city_name in cusines_in_city.keys():
    cusines_count_in_city[city_name] = len(cusines_in_city[city_name])

df['Cusines Count In City'] = df['City'].map(cusines_count_in_city)

In [ ]:
df['Weighed Cuisines Count'] = df['Cuisines Count'] / df['Cusines Count In City']  

In [ ]:
df['Most Common Cusine in City'] = df['City'].apply(lambda x: cusines_in_city[x].most_common(1)[0][0])

In [ ]:
df['Cuisine Style'] = df.apply(lambda x: x['Cuisine Style'] if x['Cuisine Style NAN'] ==False else [x['Most Common Cusine in City']], axis=1)

In [ ]:
# зависимость места ресторана в городе от населения
df['Weighed Rank by Population'] = df['Weighed Rank']  / df['Population'] 

In [ ]:
df['PositiveWords in Reviews'] = df['PositiveWords'] / df['Number of Reviews']

In [ ]:
# Как часто в городе оставляют отзывы
df['NRP'] = df['Number of Reviews'] / df['Population']

In [ ]:
# Ранг ресторана с учетом частоты отзывов в городе
df['WRR'] =  df['Weighed Rank']  *  df['NRP'] 

In [ ]:
df['Relative Price Range'] = df['Price Range'] / df['Weighed Rank']

In [ ]:
# Средняя цена в городе
price_in_city_dict = df.groupby('City')['Price Range'].mean().to_dict()
df['Price in City'] = df['City'].map(price_in_city_dict)

In [ ]:
# Сокращаем список кухонь для анализа до N - основных, остальные Other - это почти всегда дает улучшение предсказания
N=30 #!!!

s = df['Cuisine Style'].apply(lambda x: get_cuisines(x))
slist =[]
for x in s:
    slist.extend(x)
topNcusines = set(pd.Series(slist).value_counts()[:N].index)  
df['Cuisine top N'] =df['Cuisine Style'].apply(lambda x: is_cuisine_top_N(x))

In [ ]:
# Сетевой ID ресторана (похоже, что это ID франшизы, если повторяется более 1-го раза - isNetworkRestorant)
import warnings; warnings.simplefilter('ignore')
df['Restaurant_net_id'] = df['Restaurant_id'].apply(lambda x: x.split('_')[1])
NetworkRestorants = df[df['Restaurant_net_id'].isin(df['Restaurant_net_id'].value_counts()[df['Restaurant_net_id'].value_counts()>2].index)]
NetworkRestorants['isNetworkRestorant'] = True
df['isNetworkRestorant'] = NetworkRestorants['isNetworkRestorant']
df['isNetworkRestorant'] = df['isNetworkRestorant'].fillna(False)

In [ ]:
# Помечаем, входит ли город в N-top городов, если ДА, то пишем его назавние, если НЕТ -Other
top_Cityes = df['City'].value_counts()[0:10].index.to_list()
df['TopCityes'] = df.City.apply(lambda x: x if x in top_Cityes else 'Other_City')

---
# Устранили NaN и сгенерили новые признаки(пока без dummies), посмотрим какие из них подходят 

In [ ]:
df[['Ranking', 'Rating', 'Number of Reviews', 'City', 'Price Range',
        'Restaurant_id',  'Country',
       'Restaurants Count', 'Population', 'Restaurants for Population',
       'Weighed Rank',  'Cuisines Count',
       'RevTimeDelta', 'NewestReviewDate',
       'TxtReviewsCount', 'Weighed Rank by Population', 'PositiveWords','PositiveWords in Reviews', 'WRR', 'Price in City', 'Restaurant_net_id']].hist(figsize=(20, 20), bins=100);
plt.tight_layout()

### Собираем dummies

In [ ]:
# Собираем Dummies: city, price_range, country_range, Cuisine top N

dff = pd.get_dummies(df['Cuisine top N'].apply(pd.Series).stack()).sum(level=0)
df_mcc = pd.get_dummies(df['Most Common Cusine in City'], prefix = 'MCC')
df_city = pd.get_dummies(df['City'], prefix = 'City Range')
#df_city = pd.get_dummies(df['TopCityes'], prefix = 'City', dummy_na=True)
df_price_range = pd.get_dummies(df['Price Range'], prefix = 'Price Range') 
df_country_range = pd.get_dummies(df['Country'], prefix = 'Country Range',) 
#df_season_range = pd.get_dummies(df['NewestReviewSeason'], prefix = 'Season',)

df['PositiveWordsList'] = df['PositiveWordsList'].fillna('NAN')
df_positive_words_range = pd.get_dummies(df['PositiveWordsList'].apply(pd.Series).stack(), dummy_na=False).sum(level=0)


df1 = pd.concat([df,dff], axis=1)
df1 = pd.concat([df1,df_city], axis=1)
df1 = pd.concat([df1,df_price_range], axis=1)
df1 = pd.concat([df1,df_country_range], axis=1)
df1 = pd.concat([df1,df_mcc], axis=1)
df1 = pd.concat([df1,df_positive_words_range], axis=1)

#df1 = pd.concat([df1,df_season_range], axis=1)

#df1 = pd.concat([df1,df_restid_range], axis=1)

cols_cuisine_style = dff.columns
cols_city = df_city.columns
cols_price_range =  df_price_range.columns
cols_country_range =  df_country_range.columns
cols_mcc =  df_mcc.columns
cols_positive_words = df_positive_words_range.columns
#cols_season = df_season_range.columns
#cols_restid_range =  df_restid_range.columns

In [ ]:
# Собираем признаки И Разбиваем датафрейм на части, необходимые для обучения и тестирования модели
columns = [ 'isMultyCusine', 'Price Range NAN', 'Cuisine Style NAN', 'Number of Reviews NAN', 'Number of Reviews', 'Restaurants Count', 'Cuisines Count', 'RevTimeDelta',        'NewestReviewDate', 'PositiveWords',  'Weighed Rank', 'Ranking', 'Weighed Rank by Population',  'Cusines Count In City', 'WRR', 'ID_TA', 'Weighed Rank min max', 'Price in City',  'Cuisine Style NAN', 'isCapital', 'Population', 'Restaurants for Population',  ]

#columns = [ 'isMultyCusine', 'Price Range NAN', 'Cuisine Style NAN', 'Number of Reviews NAN', 'Number of Reviews', 'Restaurants Count', 'Cuisines Count', 'RevTimeDelta',        'NewestReviewDate', 'PositiveWords',  'Weighed Rank', 'Ranking', 'Weighed Rank by Population',  'Cusines Count In City', 'WRR', 'ID_TA', 'Weighed Rank min max', 'Price in City', 'Weighed Rank by Population',  ]

columns.extend(cols_price_range.tolist())
columns.extend(cols_cuisine_style.tolist())
columns.extend(cols_city.tolist())
columns.extend(cols_country_range.tolist())

columns.extend(cols_positive_words.tolist())

#Разбиваем датафрейм на части, необходимые для обучения и тестирования модели
X = df1[df1.ForTrain][columns]

y = df1[df1.ForTrain]['Rating']

In [ ]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [ ]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 20% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

# Model 


In [ ]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [ ]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
regr = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [ ]:
# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [ ]:
y_pred_old = y_pred.copy()
y_pred = round_of_rating(y_pred) 

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred),  metrics.mean_absolute_error(y_test, y_pred_old) )

In [ ]:
#Вычисляем коэффициент детерминации:
R_2 = metrics.r2_score(y_test, y_pred)
print(R_2)

In [ ]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(regr.feature_importances_, index=X.columns)
feat_importances.nlargest(30).plot(kind='barh');

# Submission
Если все устраевает - готовим Submission на кагл

In [ ]:
# Предсказываем рейтинги на датасете для предсказаний (ForTrain == False)
X_submission = df1[df1.ForTrain == False][columns]
y_pred_submission = round_of_rating(regr.predict(X_submission))

In [ ]:
# Формируем датасет с предсказаниями Restaurant_id -- Rating
submission_df = pd.DataFrame()
submission_df['Restaurant_id'] = df1[df1.ForTrain == False]['Restaurant_id']
submission_df['Rating'] = y_pred_submission
submission_df.head(15)

-----------------
# Проверяем годность предсказаний  
Распределения достаточно похожи - ОК!

In [ ]:
print('Тренировочный датасет')
df[df.ForTrain].Rating.value_counts()

In [ ]:
print('Предсказания масштабированные в 4 раза')
submission_df.Rating.value_counts()*4

In [ ]:
# Сохраняем предсказания
submission_df.to_csv('submission.csv', index=False)